In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BechmarkTools.Trial: 263 samples with 1 evaluations.
 Range (min … max):  13.883 ms … 36.968 ms  ┊ GC (min … max):  0.00% … 42.03%
 Time  (median):     16.919 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   18.997 ms ±  4.415 ms  ┊ GC (mean ± σ):  12.80% ± 15.62%

       ▃▄▆█▂                                                   
  ▄▆▆▅▇██████▇▆▆▅▃▃▄▃▃▃▃▃▁▃▃▄▃▅▅▃▄▄▄▃▆▃▄▄▃▄▄▄▃▃▄▁▃▃▃▃▁▁▃▁▁▁▁▃ ▃
  13.9 ms         Histogram: frequency by time        31.2 ms <

 Memory estimate: 12.28 MiB, allocs estimate: 206592.

In [6]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9705694242806088, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=1.9705694242806089, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=2.970569424280608, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=3.9705694242806078, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=4.970569424280607, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.970569424280608, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=6.970569424280608, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=7.970569424280609, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=8.970569424280608, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=9.97056942428061, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.970569424280608, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64